In [1]:
from toMatch import *

C:\Users\dongm\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


# Import ProductBarcode

In [2]:
mydir = 'C:/Users/dongm/COSMOCHAIN/Data/Product/상공회의소/'
dir_output = 'C:/Users/dongm/COSMOCHAIN/Data/'

jsonFileNameList = getJsonFileNameList(mydir)
print('파일 목록')
print(jsonFileNameList)

파일 목록
['ProductBarcode.json']


In [3]:
dflist=[]
for jsonFile in jsonFileNameList:
    print('change <',jsonFile, '> to Dataframe')
    dflist.append(json2dataframe(mydir,jsonFile))
    
    

change < ProductBarcode.json > to Dataframe


In [4]:
barcode = dflist[0]
barcode = barcode[['BARCODE','PRNM','CLASSCODE','CLASSNAME4']]
# 바코드 국가번호 큰 애들부터 분류시작
barcode = barcode.sort_values(by=['BARCODE'],ascending=False)
barcode = barcode.reset_index(drop=True)
display(barcode.head())


,BARCODE,PRNM,CLASSCODE,CLASSNAME4
0,9909208059210,쥬리아 알로에 베라 수딩젤,03209999,기타화장품
1,9809169793317,다이알 선물세트,03200205,핸드워시
2,9771210107001,"이스틴 틴트 루즈 1,2",03200499,기타아이메이크업용품
3,9556006060247,JB Bedtime bath 1000ml,03200199,기타목욕용화장품
4,9556006011058,PH5.5 250ML,03200199,기타목욕용화장품


# Import 상품데이터 Master

In [5]:
mydir = 'C:/Users/dongm/COSMOCHAIN/Data/Product/크롤링/'
filename = '상품데이터 Master Final.xlsx'
df = pd.read_excel(mydir + filename, sheet_name = 3)


In [8]:
master = df.copy() #[['브랜드명 (한글)','상품명 (한글)','원상품명']]
master.columns

Index(['브랜드명 (한글)', '브랜드명 (영문)', '상품명 (한글)', '색상', '타입', '사이트 기준 카테고리',
       '코스모체인 기준 카테고리', 'URL', '이미지', '용량', '원상품명', '상품명 (영문)', 'Status',
       '제품 이미지', '정상가', '판매가', '사이트 기준 pid', '한정판 유무', '88code', 'Unnamed: 19',
       'notes'],
      dtype='object')

# Match!

In [9]:
master = perfectMatch(master, barcode, test=False)

--- 2019-01-14 17:21:13.027174 ---
--- Master Data 개수 : 18048 ---
0.0 퍼센트 진행중
--- 예상 소요 시간 : 25.44 minutes ---
2.0 퍼센트 진행중
4.0 퍼센트 진행중
6.0 퍼센트 진행중
8.0 퍼센트 진행중
10.0 퍼센트 진행중
12.0 퍼센트 진행중
14.0 퍼센트 진행중
16.0 퍼센트 진행중
18.0 퍼센트 진행중
20.0 퍼센트 진행중
22.0 퍼센트 진행중
24.0 퍼센트 진행중
26.0 퍼센트 진행중
28.0 퍼센트 진행중
30.0 퍼센트 진행중
32.0 퍼센트 진행중
34.0 퍼센트 진행중
36.0 퍼센트 진행중
38.0 퍼센트 진행중
40.0 퍼센트 진행중
42.0 퍼센트 진행중
44.0 퍼센트 진행중
46.0 퍼센트 진행중
48.0 퍼센트 진행중
50.0 퍼센트 진행중
52.0 퍼센트 진행중
54.0 퍼센트 진행중
56.0 퍼센트 진행중
58.0 퍼센트 진행중
60.0 퍼센트 진행중
62.0 퍼센트 진행중
64.0 퍼센트 진행중
66.0 퍼센트 진행중
68.0 퍼센트 진행중
70.0 퍼센트 진행중
72.0 퍼센트 진행중
74.0 퍼센트 진행중
76.0 퍼센트 진행중
78.0 퍼센트 진행중
80.0 퍼센트 진행중
82.0 퍼센트 진행중
84.0 퍼센트 진행중
86.0 퍼센트 진행중
88.0 퍼센트 진행중
90.0 퍼센트 진행중
92.0 퍼센트 진행중
94.0 퍼센트 진행중
96.0 퍼센트 진행중
98.0 퍼센트 진행중
100.0 퍼센트 진행중
--- Match End : 2019-01-14 17:46:57.468881 ---
--- 전체 소요 시간 : 25.74 minutes ---


# Export matched

In [12]:
matchOutputFilename = 'matched 114.xlsx'

writer = pd.ExcelWriter(dir_output + matchOutputFilename, 
                        engine='xlsxwriter',options={'strings_to_urls': False})
master.to_excel(writer, sheet_name='Sheet1')
writer.close()

# 매칭 개수 확인

In [13]:
%%capture cap --no-stderr
matchingResult(master)

In [14]:
with open(dir_output+matchOutputFilename[:-4]+'txt', 'w') as out: out.write(cap.stdout)

# Not Matched T_T

In [15]:
master2 = notMatched(master)

# Export not matched

In [16]:
notMatchOutputFilename = 'not matched 114.xlsx'
writer = pd.ExcelWriter(dir_output + notMatchOutputFilename, 
                        engine='xlsxwriter',options={'strings_to_urls': False})
master2.to_excel(writer, sheet_name='Sheet1')
writer.close()

# 매칭 개수 확인

In [17]:
%%capture cap --no-stderr
matchingResult(master2)

In [18]:
with open(dir_output+notMatchOutputFilename[:-4]+'txt', 'w') as out: out.write(cap.stdout)

# Additional Match

In [16]:
master3 = master2.drop(columns=['BARCODE','PRNM'])
master3 = naiveMatch(master3, barcode, test=False)

--- 2019-01-14 12:03:36.683255 ---
--- Master Data 개수 : 12333 ---
0.0 퍼센트 진행중
--- 예상 소요 시간 : 109.97 minutes ---
2.0 퍼센트 진행중
4.0 퍼센트 진행중
6.0 퍼센트 진행중
8.0 퍼센트 진행중
10.0 퍼센트 진행중
12.0 퍼센트 진행중
14.0 퍼센트 진행중
16.0 퍼센트 진행중
18.0 퍼센트 진행중
20.0 퍼센트 진행중
22.0 퍼센트 진행중
24.0 퍼센트 진행중
26.0 퍼센트 진행중
28.0 퍼센트 진행중
30.0 퍼센트 진행중
32.0 퍼센트 진행중
34.0 퍼센트 진행중
36.0 퍼센트 진행중
38.0 퍼센트 진행중
40.0 퍼센트 진행중
42.0 퍼센트 진행중
44.0 퍼센트 진행중
46.0 퍼센트 진행중
48.0 퍼센트 진행중
50.0 퍼센트 진행중
52.0 퍼센트 진행중
54.0 퍼센트 진행중
56.0 퍼센트 진행중
58.0 퍼센트 진행중
60.0 퍼센트 진행중
62.0 퍼센트 진행중
64.0 퍼센트 진행중
66.0 퍼센트 진행중
68.0 퍼센트 진행중
70.0 퍼센트 진행중
72.0 퍼센트 진행중
74.0 퍼센트 진행중
76.0 퍼센트 진행중
78.0 퍼센트 진행중
80.0 퍼센트 진행중
82.0 퍼센트 진행중
84.0 퍼센트 진행중
86.0 퍼센트 진행중
88.0 퍼센트 진행중
90.0 퍼센트 진행중
92.0 퍼센트 진행중
94.0 퍼센트 진행중
96.0 퍼센트 진행중
98.0 퍼센트 진행중
100.0 퍼센트 진행중
--- Match End : 2019-01-14 14:52:26.605682 ---
--- 전체 소요 시간 : 168.83 minutes ---


C:\Users\dongm\COSMOCHAIN\Code\DataTeam\DataProcessing\toMatch.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  master['BARCODE']=BARCODE
C:\Users\dongm\COSMOCHAIN\Code\DataTeam\DataProcessing\toMatch.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  master['PRNM']=PRNM


# 매칭 개수 확인

In [23]:
%%capture cap --no-stderr
matchingResult(master3)

In [24]:
with open(dir_output+'2'+notMatchOutputFilename[:-4]+'txt', 'w') as out: out.write(cap.stdout)

# Export additional Match

In [17]:
naiveMatchOutputFilename = 'additional matched 114.xlsx'
writer = pd.ExcelWriter(dir_output + naiveMatchOutputFilename, 
                        engine='xlsxwriter',options={'strings_to_urls': False})
master3.to_excel(writer, sheet_name='Sheet1')
writer.close()

# Load Save File

In [19]:
master = pd.read_excel(dir_output + matchOutputFilename)
master2 = pd.read_excel(dir_output + notMatchOutputFilename)


In [21]:
master2 = naiveMatch(master2, barcode, test=True)

--- 2019-01-14 10:29:52.509596 ---
--- Master Data 개수 : 18032 ---
0.0 퍼센트 진행중
--- 예상 소요 시간 : 79.62 minutes ---
--- Match End : 2019-01-14 10:30:40.283918 ---
--- 전체 소요 시간 : 0.80 minutes ---
